The linear shallow water equations are
$$\begin{align}
\partial_t s + \nabla\cdot q & = 0 \\
\partial_t q + f\hat k \times q + gH\nabla s & = 0
\end{align}$$
An exact solution is
$$s = R^{-1}(x^2 + y^2) / 2, \quad q = \frac{gH}{fR}\left(\begin{matrix}y \\ -x\end{matrix}\right).$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from tqdm.notebook import trange, tqdm
import firedrake
from firedrake import Constant, inner, perp, div, dx, ds
import irksome
from irksome import Dt

g = Constant(9.81)
f = Constant(1.0)
H = Constant(0.1)
R = Constant(H)

In [ ]:
mesh = firedrake.UnitDiskMesh(4)
fig, ax = plt.subplots()
ax.set_aspect("equal")
ax.set_axis_off()
firedrake.triplot(mesh, axes=ax);

In [ ]:
degree = 0
s_element = firedrake.FiniteElement("DG", "triangle", degree)
q_element = firedrake.FiniteElement("BDM", "triangle", degree + 1)

Q = firedrake.FunctionSpace(mesh, s_element)
V = firedrake.FunctionSpace(mesh, q_element)
Z = V * Q

In [ ]:
z = firedrake.Function(Z)

x = firedrake.SpatialCoordinate(mesh)
z.sub(0).project(g * H / (f * R) * firedrake.as_vector((-x[1], x[0])))
s_exact = (x[0]**2 + x[1]**2) / (2 * R)

s_0 = Constant(H / 5)
r = Constant(1/4)
δs = s_0 * firedrake.exp(-inner(x, x) / r**2)

z.sub(1).project(s_exact + δs);

In [ ]:
q, s = firedrake.split(z)
v, ϕ = firedrake.TestFunctions(Z)

n = firedrake.FacetNormal(mesh)

F_mass = (Dt(s) + div(q)) * ϕ * dx
F_momentum = (inner(Dt(q), v) / H + f * inner(perp(q), v) / H - g * s * div(v)) * dx
F_boundary = g * s_exact * inner(v, n) * ds
F = F_mass + F_momentum + F_boundary

In [ ]:
t = Constant(0.0)
C = np.sqrt(float(g * H))
δx = mesh.cell_sizes.dat.data_ro.min()
δt = Constant(δx / C)

method = irksome.GaussLegendre(1)
solver = irksome.TimeStepper(F, method, t, δt, z)

In [ ]:
final_time = 10.0
num_steps = int(final_time / float(δt))

zs = [z.copy(deepcopy=True)]
for step in trange(num_steps):
    solver.advance()
    zs.append(z.copy(deepcopy=True))

In [ ]:
S = firedrake.VectorFunctionSpace(mesh, "DG", degree + 1)
q = firedrake.project(z.sub(0), S)

fig, ax = plt.subplots()
ax.set_aspect("equal")
ax.set_axis_off()
colors = firedrake.quiver(q, axes=ax)
fig.colorbar(colors);

In [ ]:
dss = [firedrake.Function(Q).project(z.sub(1) - s_exact) for z in zs]

In [ ]:
ds_min = np.array([ds.dat.data_ro.min() for ds in dss]).min()
ds_max = np.array([ds.dat.data_ro.max() for ds in dss]).max()
dsm = max(-ds_min, ds_max)
print(dsm)

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
ax.set_axis_off()

kw = {"num_sample_points": 4, "vmin": -dsm, "vmax": dsm, "cmap": "managua", "shading": "gouraud"}
colors = firedrake.tripcolor(dss[0], axes=ax, **kw)
fn_plotter = firedrake.FunctionPlotter(mesh, num_sample_points=4)
def animate(ds):
    colors.set_array(fn_plotter(ds))

In [ ]:
animation = FuncAnimation(fig, animate, tqdm(dss), interval=1e3/30)

In [ ]:
HTML(animation.to_html5_video())